In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import plotly.graph_objs as go

In [2]:
df_pop = pd.read_csv('Pedestrian_Volume_Model.csv')
df_pop['latitude'] = [df_pop['point'].iloc[i].split('(')[1].split(' ')[1][:-1] for i in range(len(df_pop))] #clean the format
df_pop['longitude'] = [df_pop['point'].iloc[i].split('(')[1].split(' ')[0] for i in range(len(df_pop))]

df_ne = pd.read_csv('San_Francisco_Analysis_Neighborhoods.csv')

In [3]:
df_pop.head(2)

,OBJECTID,CNN,CNNTEXT,ST_NAME1,ST_TYPE1,ST_NAME2,ST_TYPE2,ST_NAME3,ST_TYPE3,ST_NAME4,...,SIGNALIZED,PKGMETERS,MAXPCTSLPE,MODEL6_VOL,HH_PEDMODE,PCOL_04_09,PCOL_RATE,point,latitude,longitude
0,2166,33673000,33673000,BAY SHORE,BLVD,MARENGO,ST,,,,...,0,0,4,298436,1275,0,0.0,POINT (-122.4069157431717 37.738455725335555),37.738455725335555,-122.4069157431717
1,646,23037000,23037000,JUAN BAUTIST,CIR,DIAZ,AVE,,,,...,0,0,7,728306,1804,0,0.0,POINT (-122.47717379095666 37.718014352408325),37.718014352408325,-122.47717379095666


In [4]:
df_ne.head(2)

,the_geom,NHOOD
0,MULTIPOLYGON (((-122.38157774241415 37.7530704...,Bayview Hunters Point
1,MULTIPOLYGON (((-122.40361299982803 37.7493370...,Bernal Heights


In [5]:
data = go.Scattergeo(
    lat = df_pop['latitude'], 
    lon = df_pop['longitude'], 
    locationmode='USA-states', 
    marker = dict(
        color = df_pop['MODEL6_VOL'], # color intensity depends on the pedestrian volume
        colorscale = 'Reds',
        opacity = 0.8, 
        cmin = np.min(df_pop['MODEL6_VOL']),
        cmax = np.percentile(df_pop['MODEL6_VOL'], 95), # use 95 percentile rather than the max to avoid undersaturation
        colorbar_title = 'Annual Pedetrian Number',
        symbol = 'square',
    )
)

layout = go.Layout(
    autosize = True,
    title = 'Pedestrians in San Francisco',
    geo = go.layout.Geo(
        scope = 'usa',
        center = {'lat': 37.773972, 'lon': -122.431297}, #centers on SF
        showland = True,
    )
)
choromap = go.Figure(data = [data], layout = layout)

#set the range of the plot
choromap.update_geos(lataxis_range=[37.70, 37.82], lonaxis_range=[-122.545, -122.355])

# choromap.add_trace
for j in range(len(df_ne)):
    x = []
    x = df_ne.iloc[j][0].replace(' ', ',').replace("MULTIPOLYGON,(((", '').replace(')))', '').split(',,') #clean the format
    lat, lon = [], []
    for i in range(len(x)):
        lon.append(float(x[i].split(',')[0].replace("'", '').replace('((', '').replace('))', ''))) #clean the format
        lat.append(float(x[i].split(',')[1].replace("'", '').replace('((', '').replace('))', '')))
    choromap.add_trace(go.Scattergeo(lon = lon, lat = lat, mode = 'lines', line = dict(width = 1, color = 'grey'),
                                     name = df_ne['NHOOD'][j]
                                    ))     

choromap.update_layout(showlegend=False)

# choromap.show() #interactive

choromap.show(renderer="svg")  # to render a static plot 